In [12]:
# ==========================================
# 1. IMPORTACIONES (CRÍTICO: Esto faltaba)
# ==========================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, round

# ==========================================
# 2. CONFIGURACIÓN DE LA SESIÓN
# ==========================================
# Usamos el paquete oficial de MongoDB para Spark
spark = SparkSession.builder \
    .appName("AutoInsights_ETL") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/autoinsights.raw_data") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/autoinsights.agregados") \
    .getOrCreate()

print(f"Spark Inicializado: {spark.version}")

Spark Inicializado: 3.5.0


In [13]:
# ==========================================
# 3. CARGA DE DATOS
# ==========================================
file_path = "/home/jovyan/work/data/vehicles.csv"

# inferSchema=True es cómodo pero lento. Para producción se define esquema manual.
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.cache() # Guardamos en RAM para no leer el CSV dos veces

print(f"Total inicial: {df.count()}")

Total inicial: 441802


In [14]:
# ==========================================
# 4. LIMPIEZA (TRANSFORMATION)
# ==========================================
df_clean = df.filter(
    (col("price") > 500) & 
    (col("price") < 1000000) & 
    (col("year") > 1990) & 
    (col("year") <= 2025) &
    col("manufacturer").isNotNull() &
    col("model").isNotNull()
)

print(f"Total post-limpieza: {df_clean.count()}")

Total post-limpieza: 354813


In [15]:
# ==========================================
# 5. AGREGACIÓN (TRANSFORMATION)
# ==========================================
# Calculamos KPIs agrupados
df_agg = df_clean.groupBy("manufacturer", "model", "year") \
    .agg(
        round(avg("price"), 2).alias("avg_price"),
        count("*").alias("count")
    ) \
    .orderBy("manufacturer", "model", "year")

print("Muestra de datos a guardar:")
df_agg.show(5)

Muestra de datos a guardar:
+------------+---------------+-----+---------+-----+
|manufacturer|          model| year|avg_price|count|
+------------+---------------+-----+---------+-----+
|        2015|           2016| 2014|   2013.0|  164|
|       acura|2002 rsx type s| 2002|   7900.0|    1|
|       acura|          3.0cl| 1997|   2250.0|    2|
|       acura|          3.0cl| 1999|    800.0|    1|
|       acura|  3.2 cl type s| 2003|   8389.4|    5|
+------------+---------------+-----+---------+-----+
only showing top 5 rows



In [16]:
# ==========================================
# KPI 1: PRECIO PROMEDIO POR AÑO
# ==========================================
from pyspark.sql.functions import avg, sum

df_price_year = (
    df_agg
    .groupBy("year")
    .agg(
        avg("avg_price").alias("avg_price_year")
    )
    .orderBy("year")
)

print("📈 Precio promedio por año:")
df_price_year.show(10)


📈 Precio promedio por año:
+-----+------------------+
| year|    avg_price_year|
+-----+------------------+
| 2014|            2013.0|
| 1991| 8423.258615384617|
| 1992| 8392.564722222221|
| 1993|8140.4884912280695|
| 1994| 8297.851807580175|
| 1995| 8892.037525510204|
| 1996| 7048.924289215686|
| 1997| 7247.211926229507|
| 1998| 6631.266881918816|
| 1999| 7327.859511173179|
+-----+------------------+
only showing top 10 rows



In [17]:
# ==========================================
# KPI 2: VOLUMEN DE AUTOS POR AÑO (CONFIABILIDAD)
# ==========================================
df_volume_year = (
    df_agg
    .groupBy("year")
    .agg(
        sum("count").alias("volume_year")
    )
    .orderBy("year")
)

print("📊 Volumen de autos por año:")
df_volume_year.show(10)


📊 Volumen de autos por año:
+-----+-----------+
| year|volume_year|
+-----+-----------+
| 2014|        164|
| 1991|        516|
| 1992|        497|
| 1993|        603|
| 1994|        813|
| 1995|       1035|
| 1996|       1072|
| 1997|       1467|
| 1998|       1689|
| 1999|       2607|
+-----+-----------+
only showing top 10 rows



In [18]:
# ==========================================
# KPI 3: PRECIO VS VOLUMEN (DOBLE MÉTRICA)
# ==========================================
df_price_volume = (
    df_agg
    .groupBy("year")
    .agg(
        avg("avg_price").alias("avg_price_year"),
        sum("count").alias("volume_year")
    )
    .orderBy("year")
)

print("📉📊 Precio vs Volumen por año:")
df_price_volume.show(10)


📉📊 Precio vs Volumen por año:
+-----+------------------+-----------+
| year|    avg_price_year|volume_year|
+-----+------------------+-----------+
| 2014|            2013.0|        164|
| 1991| 8423.258615384617|        516|
| 1992| 8392.564722222221|        497|
| 1993|8140.4884912280695|        603|
| 1994| 8297.851807580175|        813|
| 1995| 8892.037525510204|       1035|
| 1996| 7048.924289215686|       1072|
| 1997| 7247.211926229507|       1467|
| 1998| 6631.266881918816|       1689|
| 1999| 7327.859511173179|       2607|
+-----+------------------+-----------+
only showing top 10 rows



In [19]:
from pyspark.sql.functions import col

# ===============================
# . DATASET PARA KILOMETRAJE
# ===============================
df_mileage = df_clean.select(
    "manufacturer",
    "model",
    "odometer",
    "price"
).filter(
    col("odometer").isNotNull() &
    (col("odometer") > 0) &
    (col("odometer") < 500000)
)

print("Ejemplo datos kilometraje:")
df_mileage.show(5)

Ejemplo datos kilometraje:
+------------+--------------------+--------+-----+
|manufacturer|               model|odometer|price|
+------------+--------------------+--------+-----+
|         gmc|sierra 1500 crew ...|   57923|33590|
|   chevrolet|      silverado 1500|   71229|22590|
|   chevrolet| silverado 1500 crew|   19160|39590|
|      toyota|tundra double cab sr|   41124|30990|
|        ford|           f-150 xlt|  128000|15000|
+------------+--------------------+--------+-----+
only showing top 5 rows



In [20]:
# ==========================================
# 6. ESCRITURA EN MONGODB (LOAD)
# ==========================================
print("Iniciando escritura en MongoDB...")

try:
    df_agg.write.format("mongo") \
        .mode("overwrite") \
        .option("database", "autoinsights") \
        .option("collection", "precios_promedio") \
        .save()
    
    print("¡ÉXITO! Datos guardados en MongoDB correctamente.")
    
except Exception as e:
    print(f"❌ Error al escribir en Mongo: {str(e)}")
    print("Tip: Verifica que el contenedor 'mongodb' esté corriendo y el conector se haya descargado.")

Iniciando escritura en MongoDB...
¡ÉXITO! Datos guardados en MongoDB correctamente.


In [21]:
# ==========================================
# . ESCRITURA EN MONGODB – KILOMETRAJE (LOAD)
# ==========================================
print("Iniciando escritura de kilometraje en MongoDB...")

try:
    df_mileage.write.format("mongo") \
        .mode("overwrite") \
        .option("database", "autoinsights") \
        .option("collection", "kilometraje") \
        .save()

    print("¡ÉXITO! Datos de kilometraje guardados en MongoDB correctamente.")

except Exception as e:
    print(f"❌ Error al escribir kilometraje en Mongo: {str(e)}")
    print("Tip: Verifica que el contenedor 'mongodb' esté corriendo y que el conector esté configurado.")


Iniciando escritura de kilometraje en MongoDB...
¡ÉXITO! Datos de kilometraje guardados en MongoDB correctamente.


In [22]:
# ==========================================
# 7. GUARDAR KPI FINAL PARA GRÁFICAS WEB
# ==========================================
df_price_volume.write.format("mongo") \
    .mode("overwrite") \
    .option("database", "autoinsights") \
    .option("collection", "kpi_price_volume") \
    .save()

print("✅ KPI price vs volume guardado en MongoDB")


✅ KPI price vs volume guardado en MongoDB


In [23]:
# Verificación usando PyMongo (Librería estándar de Python)
#!pip install pymongo
from pymongo import MongoClient
import pprint

# 1. Conectar al contenedor de Mongo
# Nota: Usamos "mongodb" porque es el nombre del servicio en docker-compose
client = MongoClient("mongodb://mongodb:27017/")

# 2. Seleccionar base de datos y colección
db = client["autoinsights"]
collection = db["precios_promedio"]

# 3. Contar documentos
total = collection.count_documents({})
print(f"📉 Total de documentos en Mongo: {total}")

# 4. Mostrar un ejemplo (tal como lo verá la API)
print("\n🔍 Ejemplo de documento guardado:")
ejemplo = collection.find_one({"manufacturer": "toyota"}) # Buscamos un Toyota como prueba
pprint.pprint(ejemplo)

📉 Total de documentos en Mongo: 48741

🔍 Ejemplo de documento guardado:
{'_id': ObjectId('697075b83702f625cddfcef6'),
 'avg_price': 38997.0,
 'count': 13,
 'manufacturer': 'toyota',
 'model': '"4runner 1-arizona owner*0-rust*new bilstein toytec lift*new '
          '33""yokohama m/t*new black rhino wheels* 3rd seat*nav*black out '
          'pkg*0-accidents"',
 'year': '2018'}
